#### `tf.Module` and it's uses in Keras

In [4]:
import tensorflow as tf

#### To get started with this, let's create a `Dense` class function in python first

In [5]:
class Dense():
  def __init__(self, in_features, out_features):
    self.w = tf.Variable(
      tf.random.normal([in_features, out_features]), name='w')
    self.b = tf.Variable(tf.zeros([out_features]), name='b')
  def __call__(self, x):
    y = tf.matmul(x, self.w) + self.b
    return tf.nn.relu(y)

This is a valid `Dense` function. We can create a custom dense layer and get the output to verify

In [21]:
dense = Dense(in_features=2, out_features=4)

In [22]:
input = tf.random.uniform((1,2))
dense(input)

<tf.Tensor: shape=(1, 4), dtype=float32, numpy=array([[0.        , 0.6987848 , 0.48126575, 0.        ]], dtype=float32)>

We have transformed the output with a Dense layer(It's not really making the vector Dense, is it :P)

Same class but inheriting from `tf.Module`

In [23]:
class Dense(tf.Module):
  def __init__(self, in_features, out_features, name=None):
    super().__init__(name=name)
    self.w = tf.Variable(
      tf.random.normal([in_features, out_features]), name='w')
    self.b = tf.Variable(tf.zeros([out_features]), name='b')
  def __call__(self, x):
    y = tf.matmul(x, self.w) + self.b
    return tf.nn.relu(y)

In [29]:
dense_new = Dense(in_features=2, out_features=4, name = "dense")

In [30]:
dense_new(input)

<tf.Tensor: shape=(1, 4), dtype=float32, numpy=array([[0.        , 0.        , 0.        , 0.89343256]], dtype=float32)>

We have the same function. (guess why the outputs are different)

**What's the point of inheriting from `tf.Module`??**

Well, there are many advantages to it. `tf.Module` helps you track variables or any modules inside it. 


It automatically collects the `trainable_variables`. This is useful in gradient descent

In [31]:
dense_new.trainable_variables

(<tf.Variable 'b:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>,
 <tf.Variable 'w:0' shape=(2, 4) dtype=float32, numpy=
 array([[-0.5385689 , -0.1550776 , -0.25981608,  0.9472819 ],
        [-0.44765753, -0.44565493,  0.11037057,  0.30974227]],
       dtype=float32)>)

#### Look at the following function

In [32]:
class SequentialModule(tf.Module):
  def __init__(self, name=None):
    super().__init__(name=name)

    self.dense_1 = Dense(in_features=3, out_features=3)
    self.dense_2 = Dense(in_features=3, out_features=2)

  def __call__(self, x):
    x = self.dense_1(x)
    return self.dense_2(x)

# You have made a model!
my_model = SequentialModule(name="the_model")

As mentioned earlier, `tf.Module` also automatically collects the modules inside it

In [33]:
module = SequentialModule(name="sequential")

In [34]:
module.submodules

(<__main__.Dense at 0x7f6ab6f29e10>, <__main__.Dense at 0x7f6b266e2050>)

### Use in Keras

`tf.keras.layers.Layer` and `tf.keras.models.Model` both are inherited and enjoy the functionality of `tf.Module`

In [36]:
issubclass(tf.keras.layers.Layer, tf.Module), issubclass(tf.keras.models.Model, tf.Module)

(True, True)

Btw, here are what `tf.keras.layers.Layer` and `tf.keras.models.Model` is commonly used for

`tf.keras.layers.Layer` - it's the class from which most of the Layers inherit from. Usually takes an input and produces one/more output

In [38]:
issubclass(tf.keras.layers.Conv2D, tf.keras.layers.Layer)

True

`tf.keras.models.Model` - it's the class which helps and includes funcnalities of training models

#### **Nice to know fact:** tf.keras.models.Model inherits from tf.keras.layers.Layer as well

In [42]:
issubclass(tf.keras.models.Model, tf.keras.layers.Layer)

True

Hence, `tf.keras.models.Model` includes functionalities of `tf.keras.layers.Layer` and other features for training, evaluating, saving, restoring models

### the `call()` function

The `call()` function can be implemented to indicate the forward pass of the layer. For an example, the `Dense` layer will calculate the Linear transormation followed by addition of bias and non-linear transformation(tanh or relu)

In [44]:
class Dense(tf.keras.layers.Layer):
  def __init__(self, in_features, out_features, name=None):
    super().__init__(name=name)
    self.w = tf.Variable(
      tf.random.normal([in_features, out_features]), name='w')
    self.b = tf.Variable(tf.zeros([out_features]), name='b')
  def call(self, x):
    y = tf.matmul(x, self.w) + self.b
    return tf.nn.relu(y)

In [45]:
dense = Dense(in_features=2, out_features=4)

In [46]:
dense(input)

<tf.Tensor: shape=(1, 4), dtype=float32, numpy=array([[2.0835488 , 0.23695548, 0.30858696, 0.        ]], dtype=float32)>

If you are wondering why you can get away with implementaing a `call()` rather than `__call__()` function, this is done by the `tf.keras.layers.Layer` class.

When you call `dense` instance, `dense.__call__()` is called, and this in turn calls the `call()` function you have implemented. This is possible because the  `__call__()` function of `tf.keras.layers.Layer` has been implemented this way

### the `build()` function

`tf.keras.layers.Layer` also offers the flexibility to wait for the weights/variables to be created until the input comes.

For an example, if you want to wait for the input to pass till you decide on the input dimension of `Dense` layer, you can do that using the `build()` function

In [53]:
class Dense(tf.keras.layers.Layer):
  def __init__(self, out_features, name=None):
    super().__init__(name=name)
    self.out_features = out_features
  
  def build(self, input_shape):
    self.w = tf.Variable(
      tf.random.normal([input_shape[-1], self.out_features]), name='w')
    self.b = tf.Variable(tf.zeros([self.out_features]), name='b')

    
  def call(self, x):
    y = tf.matmul(x, self.w) + self.b
    return tf.nn.relu(y)

In [54]:
dense = Dense(out_features=4, name="model")

In [55]:
dense(tf.random.uniform((1,3)))

<tf.Tensor: shape=(1, 4), dtype=float32, numpy=array([[0.        , 0.06571113, 0.13430151, 0.78265435]], dtype=float32)>

Again, the `build` function is called from inside the `__call__` function in `tf.keras.layers.Layer` implementation

As tf.keras.models.Model is inherited from `tf.keras.layers.Layer`, the have similar `call` and `build` function

More instruction on `tf.keras.models.Model` is available in the next notebook